# Annual Avg. WBGTmax
By Cascade Tuholske, cascade dot tuholske 1 at montana dot edu 

This script opens and stacks rasters to estimate the grid-cell average. It is set up to run on WBGTmax daily data to create the average for a given year.

To recreate Figure 1, it needs to be run on 1983 and 2016 WBGTmax data. 

The WBGTmax data can be found here: https://data.chc.ucsb.edu/people/cascade/UHE-daily/wbgtmax/

Note: Be sure to check NAN value and split strings on rasters before running; change as needed. 

In [ ]:
# Dependencies
import os
import glob
import dask
import xarray as xr
import numpy as np
import rioxarray 
import rasterio
import matplotlib.pyplot as plt
import pandas as pd
from dask.distributed import Client, LocalCluster

In [ ]:
# Files
year = '2016' #or 1983 
path = os.path.join('PATH/TO/WBGTmax')
fns = sorted(glob.glob(path + str(year) + '/*.tif'))
print(fns[0])
print(len(fns))

In [ ]:
# Start Dask Cluster 
cluster = LocalCluster(n_workers=20, threads_per_worker = 1)
client = Client(cluster)

In [ ]:
# Open rasters as Dask Arrays in an Xarray Dataset 

def open_raster(filepath):
    return rioxarray.open_rasterio(filepath, chunks='auto')

# Open the raster files individually
datasets = [open_raster(fn) for fn in fns]

# Combine the datasets
combined = xr.concat(datasets, dim='band')  # Adjust 'dim' as per your requirement

In [ ]:
# Check the data
combined

In [ ]:
# get the mean
mean = combined.mean(dim = 'band').compute()

In [ ]:
# Check the data
plt.imshow(mean.data)
plt.colorbar()

In [ ]:
mean.data

In [ ]:
# Get data to write out and write NAN values 
arr_out = np.nan_to_num(mean, nan = -9999)
arr_out

In [ ]:
# meta data
meta = rasterio.open(fns[0]).meta
meta['nodata'] = -9999
meta

In [ ]:
# write it out 
fn_out = os.path.join('PATH/OUT/GHE-wbgtmax-avg-' + str(year) + '.tif')
with rasterio.open(fn_out, 'w', **meta) as out:
    out.write_band(1, arr_out)

In [ ]:
# shut down the cluster
client.close()